<a href="https://colab.research.google.com/github/adarshakumar395/Implementation-of-RRT-Star-for-DynamicPath-Finding/blob/main/Final_Dynamically_RRT_Star_Algorithm_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Required Libraries**


In [ ]:
!pip install numpy matplotlib scipy


**Implementation of RRT Star algoritm for Dynamically Path Finding**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
import math
from scipy.spatial import KDTree
from matplotlib.animation import FuncAnimation

class Node:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        self.parent = None
        self.cost = 0.0

class Obstacle:
    def __init__(self, x, y, z, area):
        self.x = x
        self.y = y
        self.z = z
        self.area = area  # Area of the obstacle, instead of radius

class RRTStar:
    def __init__(self, start, goal, bounds, obstacles, max_iter=1000, step_size=1.0, goal_sample_rate=0.1, radius=2.0):
        self.start = Node(*start)
        self.goal = Node(*goal)
        self.bounds = bounds
        self.obstacles = [Obstacle(*obs) for obs in obstacles]
        self.max_iter = max_iter
        self.step_size = step_size
        self.goal_sample_rate = goal_sample_rate
        self.radius = radius
        self.node_list = [self.start]

    def get_random_node(self):
        if random.random() > self.goal_sample_rate:
            return Node(
                random.uniform(self.bounds[0][0], self.bounds[0][1]),
                random.uniform(self.bounds[1][0], self.bounds[1][1]),
                random.uniform(self.bounds[2][0], self.bounds[2][1])
            )
        else:
            return Node(self.goal.x, self.goal.y, self.goal.z)

    def get_nearest_node(self, rnd_node):
        node_coords = [(node.x, node.y, node.z) for node in self.node_list]
        kdtree = KDTree(node_coords)
        _, idx = kdtree.query([rnd_node.x, rnd_node.y, rnd_node.z])
        return self.node_list[idx]

    def steer(self, from_node, to_node):
        direction = np.array([to_node.x - from_node.x, to_node.y - from_node.y, to_node.z - from_node.z])
        distance = np.linalg.norm(direction)
        direction = direction / distance
        distance = min(self.step_size, distance)
        new_node = Node(
            from_node.x + direction[0] * distance,
            from_node.y + direction[1] * distance,
            from_node.z + direction[2] * distance
        )
        new_node.parent = from_node
        new_node.cost = from_node.cost + distance
        return new_node

    def is_collision_free(self, node1, node2):
        for obs in self.obstacles:
            if self.check_collision(node1, node2, obs):
                return False
        return True

    def check_collision(self, node1, node2, obs):
        direction = np.array([node2.x - node1.x, node2.y - node1.y, node2.z - node1.z])
        distance = np.linalg.norm(direction)
        direction = direction / distance
        steps = int(distance / 0.1)
        for i in range(steps):
            point = np.array([node1.x, node1.y, node1.z]) + direction * i * 0.1
            if np.linalg.norm(point - np.array([obs.x, obs.y, obs.z])) <= math.sqrt(obs.area / np.pi):
                return True
        return False

    def rewire(self, new_node, near_nodes):
        for near_node in near_nodes:
            if self.is_collision_free(new_node, near_node):
                new_cost = new_node.cost + np.linalg.norm(
                    [new_node.x - near_node.x, new_node.y - near_node.y, new_node.z - near_node.z]
                )
                if new_cost < near_node.cost:
                    near_node.parent = new_node
                    near_node.cost = new_cost

    def get_near_nodes(self, new_node):
        node_coords = [(node.x, node.y, node.z) for node in self.node_list]
        kdtree = KDTree(node_coords)
        indices = kdtree.query_ball_point([new_node.x, new_node.y, new_node.z], self.radius)
        return [self.node_list[idx] for idx in indices]

    def update_obstacles(self, new_obstacles):
        self.obstacles = [Obstacle(*obs) for obs in new_obstacles]

    def search(self):
        for i in range(self.max_iter):
            rnd_node = self.get_random_node()
            nearest_node = self.get_nearest_node(rnd_node)
            new_node = self.steer(nearest_node, rnd_node)

            if self.is_collision_free(nearest_node, new_node):
                near_nodes = self.get_near_nodes(new_node)
                self.node_list.append(new_node)
                self.rewire(new_node, near_nodes)

            if i % 100 == 0:
                self.plot_tree()
                plt.pause(0.01)

            if self.is_collision_free(new_node, self.goal):
                self.goal.parent = new_node
                self.goal.cost = new_node.cost + np.linalg.norm(
                    [new_node.x - self.goal.x, new_node.y - self.goal.y, new_node.z - self.goal.z]
                )
                self.node_list.append(self.goal)
                return self.get_final_path()

        return None

    def get_final_path(self):
        path = []
        node = self.goal
        while node.parent is not None:
            path.append((node.x, node.y, node.z))
            node = node.parent
        path.append((self.start.x, self.start.y, self.start.z))
        return path[::-1]

    def plot_tree(self):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.set_xlim(self.bounds[0])
        ax.set_ylim(self.bounds[1])
        ax.set_zlim(self.bounds[2])

        for node in self.node_list:
            if node.parent:
                ax.plot(
                    [node.x, node.parent.x],
                    [node.y, node.parent.y],
                    [node.z, node.parent.z],
                    "g-"
                )

        for obs in self.obstacles:
            self.plot_sphere(ax, (obs.x, obs.y, obs.z), math.sqrt(obs.area / np.pi))
            ax.text(obs.x, obs.y, obs.z, f'Obstacle {obs.x, obs.y, obs.z}', color='red', fontsize=10)

        plt.plot(self.start.x, self.start.y, self.start.z, "ro", label='Start')
        plt.plot(self.goal.x, self.goal.y, self.goal.z, "bo", label='Goal')

        ax.legend()
        plt.show()

    def plot_path(self):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.set_xlim(self.bounds[0])
        ax.set_ylim(self.bounds[1])
        ax.set_zlim(self.bounds[2])

        path = self.get_final_path()
        path_x, path_y, path_z = zip(*path)
        ax.plot(path_x, path_y, path_z, linewidth=3, label='Shortest Path', color='cyan')

        ax.scatter(self.start.x, self.start.y, self.start.z, color='red', s=100, label='Start')
        ax.scatter(self.goal.x, self.goal.y, self.goal.z, color='blue', s=100, label='Goal')

        for obs in self.obstacles:
            self.plot_sphere(ax, (obs.x, obs.y, obs.z), math.sqrt(obs.area / np.pi))

        ax.legend()
         # Print coordinates of the shortest path
        print("Coordinates of Shortest Path:")
        for coord in path:
            print(f"({coord[0]}, {coord[1]}, {coord[2]})")
        plt.show()

    def animate_path(self):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.set_xlim(self.bounds[0])
        ax.set_ylim(self.bounds[1])
        ax.set_zlim(self.bounds[2])

        path = self.get_final_path()
        path_x, path_y, path_z = zip(*path)
        shortest_path, = ax.plot([], [], [], linewidth=3, label='Shortest Path', color='cyan')
        drone, = ax.plot([], [], [], marker='^', markersize=10, color='green', label='Drone')

        ax.scatter(self.start.x, self.start.y, self.start.z, color='red', s=100, label='Start')
        ax.scatter(self.goal.x, self.goal.y, self.goal.z, color='blue', s=100, label='Goal')

        for obs in self.obstacles:
            self.plot_sphere(ax, (obs.x, obs.y, obs.z), math.sqrt(obs.area / np.pi))

        ax.legend()

        def update_frame(num):
            drone.set_data(path_x[num], path_y[num])
            drone.set_3d_properties(path_z[num])
            shortest_path.set_data(path_x[:num+1], path_y[:num+1])
            shortest_path.set_3d_properties(path_z[:num+1])
            return drone, shortest_path

        ani = FuncAnimation(fig, update_frame, frames=len(path_x), blit=True)
        plt.show()

    def plot_sphere(self, ax, center, radius):
        u = np.linspace(0, 2 * np.pi, 100)
        v = np.linspace(0, np.pi, 100)
        x = center[0] + radius * np.outer(np.cos(u), np.sin(v))
        y = center[1] + radius * np.outer(np.sin(u), np.sin(v))
        z = center[2] + radius * np.outer(np.ones(np.size(u)), np.cos(v))
        ax.plot_surface(x, y, z, color='r', alpha=0.3)

def main():
    start_x = float(input("Enter x-coordinate of start point: "))
    start_y = float(input("Enter y-coordinate of start point: "))
    start_z = float(input("Enter z-coordinate of start point: "))

    goal_x = float(input("Enter x-coordinate of goal point: "))
    goal_y = float(input("Enter y-coordinate of goal point: "))
    goal_z = float(input("Enter z-coordinate of goal point: "))

    num_obstacles = int(input("Enter number of obstacles: "))
    obstacles = []
    for i in range(num_obstacles):
        obs_x = float(input(f"Enter x-coordinate of obstacle {i + 1}: "))
        obs_y = float(input(f"Enter y-coordinate of obstacle {i + 1}: "))
        obs_z = float(input(f"Enter z-coordinate of obstacle {i + 1}: "))
        obs_area = float(input(f"Enter area of obstacle {i + 1}: "))
        obstacles.append((obs_x, obs_y, obs_z, obs_area))

    bounds = [(0, 100), (0, 100), (0, 100)]  # Example bounds, adjust as per your needs

    rrt_star = RRTStar(start=(start_x, start_y, start_z),
                       goal=(goal_x, goal_y, goal_z),
                       bounds=bounds,
                       obstacles=obstacles)

    # Search for the path
    path = rrt_star.search()

    if path is None:
        print("No path found!")
    else:
        # Plotting the final path and animating the drone
        rrt_star.plot_path()
        rrt_star.animate_path()

if __name__ == "__main__":
    main()
